In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random

In [2]:
all_a = []
limit = 30
for i in tqdm(range(0, 100000, limit)):
    if i == 0:
        i = ''
    else:
        i = f'/+{i}'
    url = f'https://forum.lowyat.net/PropertyTalk{i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href') and a_.get('href').startswith('/topic')]
    a = list(set([a_ for a_ in a if len(a_.split('/')) == 3]))
    if not len(a):
        break
    all_a.extend(a)

 23%|█████████▏                              | 767/3334 [10:51<36:21,  1.18it/s]


In [3]:
all_a = list(set(all_a))

In [6]:
len(all_a)

22966

In [7]:
with open('PropertyTalk-topics.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_text(url):
    texts = []
    try:
        r_ = requests.get(url)
        soup = BeautifulSoup(r_.content, "lxml")
        divs = soup.find_all('div', {'class': 'post_text'})
        for div in divs:
            t = BeautifulSoup(str(div).replace('<br/>', '\n')).text.strip()
            if len(t):
                texts.append(t)
    except Exception as e:
        pass
    return texts

In [ ]:
limit = 20
worker_size = 1
per_batch = worker_size * limit

for topic in tqdm(sorted(all_a)):
    
    base_dir = topic[1:].replace('/', '-')
    os.makedirs(base_dir, exist_ok = True)
    os.makedirs('done-SeriousKopitiam', exist_ok = True)
    
    base_dir_filename = os.path.join('done-SeriousKopitiam', f'{base_dir}.json')
    
    if os.path.exists(base_dir_filename):
        continue
    
    need_break = False
    last_texts = []
    for i in range(0, 100000, per_batch):
        filename = os.path.join(base_dir, f'{i}.json')
        if os.path.exists(filename):
            continue
        aranged = np.arange(i, i + per_batch, limit)
        urls = []
        for a_ in aranged:
            if a_ == 0:
                a_ = ''
            else:
                a_ = f'/+{a_}'
            url = f'https://forum.lowyat.net/{topic}{a_}'
            urls.append(url)
        
        texts = []
        
        with ThreadPoolExecutor(max_workers=worker_size) as executor:
            futures = {executor.submit(get_text, item): item for item in urls}
            
            for future in as_completed(futures):
                r = future.result()
                texts.extend(r)
                if not len(r):
                    need_break = True
                    
                s = set(texts)
                if len(set(last_texts) & s) == len(s):
                    need_break = True
                    
                last_texts = texts[:]
        
        with open(filename, 'w') as fopen:
            json.dump(texts, fopen)
            
        time.sleep(random.uniform(0.5, 1.5))
        
        if need_break:
            break
    
    with open(base_dir_filename, 'w') as fopen:
        json.dump({'status': True}, fopen) 

  5%|█▌                               | 1094/22966 [2:05:17<30:41:14,  5.05s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
  6%|██                               | 1419/22966 [2:45:16<27:41:49,  4.63s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
  8%|██▍                              | 1736/22966 [3:28:

 46%|██████████████▏                | 10484/22966 [27:26:13<19:28:12,  5.62s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 46%|██████████████▏                | 10494/22966 [27:27:07<16:22:43,  4.73s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 46%|██████████████▏                | 10535/22966 [27:33:57<15:37:28,  4.52s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, byt

 47%|██████████████▌                | 10752/22966 [28:00:56<13:37:44,  4.02s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 47%|██████████████▌                | 10780/22966 [28:04:08<17:19:54,  5.12s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input err

encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 56%|█████████████████▏             | 12775/22966 [34:00:17<12:18:08,  4.35s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to in

 66%|█████████████████████▏          | 15223/22966 [41:23:14<9:11:35,  4.27s/it]encoding error : input conversion failed due to input error, bytes 0x81 0xBE 0x2D 0xE9
encoding error : input conversion failed due to input error, bytes 0x81 0xBE 0x2D 0xE9
encoding error : input conversion failed due to input error, bytes 0x81 0xBE 0x2D 0xE9
 67%|█████████████████████▍          | 15374/22966 [41:52:36<9:00:02,  4.27s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 68%|█████████████████████          | 15632/22966 [42:57:53<17:28:40,  8.58s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x20 0x6C 0x65
encoding error : input conversion failed due to input error, bytes 0x8D 0x20 0x6C 0x65
encoding error : input conversion failed due to input error, byt

encoding error : input conversion failed due to input error, bytes 0x81 0x3C 0x62 0x72
encoding error : input conversion failed due to input error, bytes 0x81 0x3C 0x62 0x72
encoding error : input conversion failed due to input error, bytes 0x81 0x3C 0x62 0x72
 73%|██████████████████████▍        | 16657/22966 [47:09:43<13:37:50,  7.78s/it]encoding error : input conversion failed due to input error, bytes 0x90 0x8C 0xE6 0xAD
encoding error : input conversion failed due to input error, bytes 0x90 0x8C 0xE6 0xAD
encoding error : input conversion failed due to input error, bytes 0x90 0x8C 0xE6 0xAD
 73%|██████████████████████▌        | 16702/22966 [47:24:09<61:54:22, 35.58s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xE8 0xBD 0xA6
encoding error : input conversion failed due to input error, bytes 0x9E 0xE8 0xBD 0xA6
encoding error : input conversion failed due to input error, bytes 0x9E 0xE8 0xBD 0xA6
 73%|██████████████████████▋        | 16841/22966 [47:54:

 82%|██████████████████████████▏     | 18835/22966 [52:53:51<5:52:50,  5.12s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 82%|██████████████████████████▎     | 18858/22966 [52:55:59<5:36:25,  4.91s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 82%|█████████████████████████▌     | 18940/22966 [53:08:19<17:08:16, 15.32s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, byt

In [11]:
!mkdir PropertyTalk

In [12]:
!mv topic-* PropertyTalk

In [13]:
!du -hs PropertyTalk

903M	PropertyTalk


In [15]:
# !zip -r PropertyTalk.zip PropertyTalk

In [16]:
from glob import glob
texts = []

files = glob('PropertyTalk/topic-*/*.json')
for f in tqdm(files):
    with open(f) as fopen:
        texts.extend(json.load(fopen))

100%|████████████████████████████████| 108309/108309 [00:02<00:00, 46905.09it/s]


In [17]:
len(texts)

1643788

In [18]:
texts = list(set(texts))
len(texts)

1474838

In [19]:
with open('PropertyTalk.json', 'w') as fopen:
    json.dump(texts, fopen)